In [ ]:
import hpmcm
import tables_io
import glob
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
DATADIR = "shear_tract"
shear_st = "0p01"
SOURCE_TABLEFILES = sorted(glob.glob(os.path.join(DATADIR, f"shear_wmom_{shear_st}_uncleaned_*.pq")))
SOURCE_TABLEFILES.reverse()
VISIT_IDS = np.arange(len(SOURCE_TABLEFILES))

REF_DIR = (37.91729, 6.94389)  # RA, DEC in deg
PIXEL_R2CUT = 2

In [ ]:
matcher = hpmcm.Match.createCoaddCellsForTract(pixelR2Cut=PIXEL_R2CUT)

In [ ]:
matcher.reduceData(SOURCE_TABLEFILES, VISIT_IDS)

In [ ]:
matcher.fullData[0].xCell_coadd.max()

In [ ]:
matcher.analysisLoop()

In [ ]:
oDict = matcher.analyzeCell(50, 180, True)

In [ ]:
cellData = oDict['cellData']
cd = cellData._clusterDict
od = cellData._objectDict
all_clusters = list(cd.keys())
idx = list(cd.keys())[0]
image = oDict['image']
cluster = cd[idx]

In [ ]:
nsrcs = []

ideal = []
mixed = []
edge_mixed = []
edge_missing = []
edge_extra = []

missing = []
two_missing = []
three_missing = []
single = []
extra = []
faint = []

cut1 = []
cut2 = []

for iC, cellData in matcher._clusters.items():

    cd = cellData._clusterDict

    for k, c in cd.items():
        edge_case = False
        if (np.fabs(c.data.xCell_coadd) > 75).all() or (np.fabs(c.data.yCell_coadd) > 75).all():
            cut1.append(k)
            continue
        if np.fabs(c.data.xCell_coadd.mean()) > 75 or np.fabs(c.data.yCell_coadd.mean()) > 75:
            cut2.append(k)
            continue
            
        nsrcs.append(c._nSrc)    
            
        if (np.fabs(c.data.xCell_coadd) > 73).any() or (np.fabs(c.data.yCell_coadd) > 73).any():
            edge_case = True
        if c._nSrc == c._nUnique and c._nSrc == 5:
            ideal.append(k)
        elif c._nSrc == 5 and c._nUnique != 5 and edge_case:
            edge_mixed.append(k)
        elif c._nSrc == 5 and c._nUnique != 5:
            mixed.append(k)
        elif c._nSrc < 5 and edge_case:
            edge_missing.append(k)
        elif c._nSrc > 5 and edge_case:
            edge_extra.append(k)    
        elif c._nSrc < 5 and (c.data.SNR < 7.5).any():
            faint.append(k)
        elif c._nSrc == 4:
            missing.append(k)
        elif c._nSrc == 3:
            two_missing.append(k)
        elif c._nSrc == 2:
            three_missing.append(k)
        elif c._nSrc < 2:
            single.append(k)
        elif c._nSrc > 5:
            extra.append(k)


In [ ]:
print("All clusters:                                  ", len(nsrcs))
print("good (5 source from 5 catalogs):               ", len(ideal))
print("mixed (5 source from < 5 catalogs):            ", len(mixed))
print("edge_mixed (mixed near edge of cell):          ", len(edge_mixed))
print("edge_missing (< 5 sources, near edge of cell): ", len(edge_missing))
print("edge_extra (> 5 sources, near edge of cell):   ", len(edge_extra))
print("faint (< 5 sources, SNR < 5):                  ", len(faint))
print("one missing (4 sources, not near edge):        ", len(missing))
print("two missing (3 sources, not near edge):        ", len(two_missing))
print("three missing (2 sources, not near edge):      ", len(three_missing))
print("single  (1 source, not near edge):             ", len(single))
print("extra (> 5 sources, not near edge):            ", len(extra))
print("cut1                                           ", len(cut1))
print("cut2                                           ", len(cut2))

In [ ]:
obj_nsrcs = []
for k, c in od.items():
    obj_nsrcs.append(c._nSrc)    

obj_faint_snr = []
obj_ideal = []
obj_ideal_faint = []
obj_mixed = []
obj_edge_mixed = []
obj_edge_missing = []
obj_edge_extra = []

obj_missing = []
obj_two_missing = []
obj_three_missing = []
obj_single = []
obj_orphan = []
obj_extra = []
obj_faint = []

for k, c in od.items():
    edge_case = False
    if (np.fabs(c._data.xCell_coadd) > 73).any() or (np.fabs(c._data.yCell_coadd) > 73).any():
        edge_case = True
    if c._nSrc == c._nUnique and c._nSrc == 5 and (c._data.SNR < 7.5).any():
        obj_ideal_faint.append(k)
    elif c._nSrc == c._nUnique and c._nSrc == 5:
        obj_ideal.append(k)
    elif c._nSrc == 5 and c._nUnique != 5 and edge_case:
        obj_edge_mixed.append(k)
    elif c._nSrc == 5 and c._nUnique != 5:
        obj_mixed.append(k)
    elif c._nSrc < 5 and edge_case:
        obj_edge_missing.append(k)
    elif c._nSrc > 5 and edge_case:
        obj_edge_extra.append(k)    
    elif c._nSrc < 5 and (c._data.SNR < 7.5).any():
        obj_faint_snr.append(c._data.SNR.mean())
        obj_faint.append(k)
    elif c._nSrc < 5 and c._parentCluster._nSrc >= 5:
        obj_orphan.append(k)
    elif c._nSrc == 4:
        obj_missing.append(k)
    elif c._nSrc == 3:
        obj_two_missing.append(k)
    elif c._nSrc == 2:
        obj_three_missing.append(k)
    elif c._nSrc < 2:
        obj_single.append(k)
    elif c._nSrc > 5:
        obj_extra.append(k)


In [ ]:
print("All Objects:                                   ", len(od))
print("good (5 source from 5 catalogs):               ", len(obj_ideal))
print("good faint                                     ", len(obj_ideal_faint))
print("mixed (5 source from < 5 catalogs):            ", len(obj_mixed))
print("edge_mixed (mixed near edge of cell):          ", len(obj_edge_mixed))
print("edge_missing (< 5 sources, near edge of cell): ", len(obj_edge_missing))
print("edge_extra (> 5 sources, near edge of cell):   ", len(obj_edge_extra))
print("faint (< 5 sources, SNR < 5):                  ", len(obj_faint))
print("orphan (split off from larger cluster          ", len(obj_orphan))
print("one missing (4 sources, not near edge):        ", len(obj_missing))
print("two missing (3 sources, not near edge):        ", len(obj_two_missing))
print("three missing (2 sources, not near edge):      ", len(obj_three_missing))
print("single  (1 source, not near edge):             ", len(obj_single))
print("extra (> 5 sources, not near edge):            ", len(obj_extra))

In [ ]:
def showObjects(img, cluster, cellData):
    extent = (0, cluster._footprint.getBBox().getWidth(),
              0, cluster._footprint.getBBox().getHeight())
    cluster.extract(cellData)
    xOffset = cellData._minPix[0] + cluster._footprint.getBBox().getBeginY()
    yOffset = cellData._minPix[1] + cluster._footprint.getBBox().getBeginX()
    xOff = cluster.data.xPix - xOffset
    yOff = cluster.data.yPix - yOffset
    catIndices = cluster.catIndices
    img = plt.imshow(image[cluster._footprint.getBBox()].array, origin='lower', extent=extent)
    cb = plt.colorbar()
    colors = ['red', 'blue', 'green', 'cyan', 'orange']
    markers = ['.', '<', '>', 'v', '^']
    for iObj, obj in enumerate(cluster.objects):
        xC = obj._xCent - xOffset
        yC = obj._yCent - yOffset
        for x_, y_, i_ in zip(yOff[obj._mask], xOff[obj._mask], catIndices[obj._mask]):
            img.axes.scatter(x_, y_, c=colors[iObj%5], marker=markers[i_ % 5])
    return img.axes.figure


In [ ]:
_ = showObjects(image, od[obj_edge_extra[0]]._parentCluster, cellData)

In [ ]:
od[obj_edge_extra[0]]._parentCluster.data